In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
spark = SparkSession.builder.appName("data_processing").getOrCreate()


In [ ]:
# give the

In [2]:
data_dir = "./src_data"
geo_ip = spark.read.option("header", "true").option('delimiter', ';').csv(f"{data_dir}/Aux_1A_Geolocation.csv")
geo_ip.schema.fields

[StructField(_c0,StringType,true),
 StructField(ANIP,StringType,true),
 StructField(Lat,StringType,true),
 StructField(Lon,StringType,true),
 StructField(Country,StringType,true),
 StructField(Region,StringType,true),
 StructField(City,StringType,true),
 StructField(TZ,StringType,true),
 StructField(ASN,StringType,true),
 StructField(ISP,StringType,true)]

In [3]:
passive_dns = spark.read.csv(f"{data_dir}/Aux_2_PassiveDNS.csv", header=True, inferSchema=True)
passive_dns.schema.fields

[StructField(ip,StringType,true),
 StructField(numrecords,IntegerType,true),
 StructField(avgdomainsinrecords,DoubleType,true),
 StructField(stddevdomainsinrecords,DoubleType,true),
 StructField(maxdomainsinrecords,IntegerType,true),
 StructField(mediandomainsinrecords,DoubleType,true),
 StructField(avglenrecords,DoubleType,true),
 StructField(stddevlenrecords,DoubleType,true),
 StructField(maxlenrecords,IntegerType,true),
 StructField(medianlenrecords,DoubleType,true),
 StructField(avgsimilarityrecords,DoubleType,true),
 StructField(stddevsimilarityrecords,DoubleType,true),
 StructField(maxsimilarityrecords,DoubleType,true),
 StructField(mediansimilarityrecords,DoubleType,true),
 StructField(avgentropyrecords,DoubleType,true),
 StructField(stddeventropyrecords,DoubleType,true),
 StructField(maxentropyrecords,DoubleType,true),
 StructField(medianentropyrecords,DoubleType,true),
 StructField(avgmaxconsecutivecharsrecords,DoubleType,true),
 StructField(stddevmaxconsecutivecharsrecords,Do

In [4]:
dataset = spark.read.json(f"./src_data/dataset.idea/dataset.idea")

In [5]:
from data_processing.utils.util import pre_process_dataset

dataset = pre_process_dataset(dataset)

In [6]:
dataset.count()

4875902

In [18]:
from data_processing.utils.util import get_source_data

geo_enriched_data_src = get_source_data(dataset, geo_ip, include_long_lat=True, include_region=True)


In [19]:
from data_processing.utils.util import get_target_data

geo_enriched_data_tgt = get_target_data(dataset, geo_ip, include_long_lat=True, include_region=True)


In [20]:
geo_enriched_data_tgt = geo_enriched_data_tgt.select("ID", "EventTime", "TargetCountry", "TargetCountry", "TargetLongitude", "TargetRegion")
geo_enriched_data_src = geo_enriched_data_src.select("ID", "EventTime", "SourceCountry", "SourceLongitude", "SourceLatitude", "SourceRegion")

In [21]:
geo_enriched_data = geo_enriched_data_src.join(geo_enriched_data_tgt, on="ID", how="inner")
geo_enriched_data.show()

+--------------------+-------------+---------------+--------------+-----------------+-------------------+-------------+-------------+---------------+-------------+
|                  ID|SourceCountry|SourceLongitude|SourceLatitude|     SourceRegion|          EventTime|TargetCountry|TargetCountry|TargetLongitude| TargetRegion|
+--------------------+-------------+---------------+--------------+-----------------+-------------------+-------------+-------------+---------------+-------------+
|0001a9a1-76fe-49f...|           CN|       117.0225|       25.1072|           Fujian|2019-03-12 09:42:03|           CZ|           CZ|        15.5833|Kraj Vysocina|
|00026cdf-9e78-46b...|           FR|         6.0637|       48.7497|        Grand Est|2019-03-15 00:53:04|           CZ|           CZ|        15.5833|Kraj Vysocina|
|00044461-b455-4fc...|           RU|        66.6019|         66.53|    Yamalo-Nenets|2019-03-15 14:32:39|           CZ|           CZ|        14.4378|        Praha|
|00051a95-d196-4